In [8]:
import numpy as np
from keras import Sequential
from keras.layers import LSTM, Dense, Embedding, SpatialDropout1D


In [24]:
"""
This is a script which will check for non-random subsequences 
from the quantum random number generator onboard the CubeSat. 
The code will first perform a benchmark suite of tests known 
as Diehard tests. We will then use the results from these 
tests in order to train and compare the results of a 
neural network approach (LSTMs).
"""

# Define important global variables here
tot_num_samples = 2000 # total number of training samples (50/50 split between rand and non-rand)
train_split = int(int(tot_num_samples/2) * 0.9) # use 90% for training, 10% for testing
bitstream_len = 32     # length of observation window of network
batch_size = 8         # batch size (how many samples NN sees during each iteration)

# Generate a set of pseudo random numbers
pseudo_rand_data_train = np.random.randint(low=0,high=2,size=(train_split,bitstream_len))
pseudo_rand_data_test = np.random.randint(low=0,high=2,size=(int(tot_num_samples/2) - train_split,bitstream_len))
pseudo_rand_labels_train = np.ones(train_split)
pseudo_rand_labels_test = np.ones(int(tot_num_samples/2) - train_split)

# Load in set of true random numbers
true_rand_data_train = np.loadtxt('true_random_nums.txt')[:train_split,:]
true_rand_data_test = np.loadtxt('true_random_nums.txt')[train_split:int(tot_num_samples/2),:]
true_rand_labels_train = np.zeros(train_split)
true_rand_labels_test = np.zeros(int(tot_num_samples/2) - train_split)

# Combine all data and labels into two arrays (x and y respectively)
X_train = np.concatenate((pseudo_rand_data_train,true_rand_data_train))
X_test = np.concatenate((pseudo_rand_data_test,true_rand_data_test))
Y_train = np.concatenate((pseudo_rand_labels_train,true_rand_labels_train))
Y_test = np.concatenate((pseudo_rand_labels_test,true_rand_labels_test))

# Randomly shuffle training sets
idx_shuffle = np.random.permutation(int(train_split*2))
X_train = X_train[idx_shuffle,:]
Y_train = Y_train[idx_shuffle]


In [25]:
print(X_test)

[[0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 [1. 1. 0. ... 1. 0. 1.]
 ...
 [0. 1. 0. ... 1. 0. 0.]
 [1. 1. 1. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 1. 1.]]


In [26]:
# Set-up LSTM network
embed_dim = 128
lstm_out = 200
batch_size = 32

model = Sequential()
model.add(Embedding(2500, embed_dim,input_length = bitstream_len))
SpatialDropout1D(0.2)
model.add(LSTM(lstm_out, dropout_U = 0.2, dropout_W = 0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'mean_squared_error', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

/home/hunter/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(200, dropout=0.2, recurrent_dropout=0.2)`
  if __name__ == '__main__':


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 32, 128)           320000    
_________________________________________________________________
lstm_7 (LSTM)                (None, 200)               263200    
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 201       
Total params: 583,401
Trainable params: 583,401
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
# Train LSTM network
model.fit(X_train, Y_train, batch_size =batch_size, epochs = 100,  verbose = 5)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [30]:
# Print score and accuracy of network. Want low score and high accuracy
score,acc = model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

Score: 0.33
Validation Accuracy: 0.51
